In [395]:
from pymongo import MongoClient
import pandas as pd
import time
import json
import seaborn as sns
import numpy as np
import os
import geopandas as gp
from cartoframes.viz import Map, Layer, popup_element
from dotenv import load_dotenv
import requests
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import math



In [2]:
client = MongoClient("localhost:27017")

In [3]:
client.list_database_names()
db = client["Ironhack"]

In [4]:
db.list_collection_names()

['PROJECT3', 'nyneigh', 'First', 'Lab', 'nyrest']

In [5]:
companys= db.get_collection("Lab")

In [6]:
companys

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack'), 'Lab')

In [7]:
result = companys.find_one()

In [8]:
df = pd.read_csv('Lab.csv')

In [9]:
data= pd.read_json ('Labjson.csv')

In [15]:
data.sample(5)

,name,category_code,number_of_employees,description,total_money_raised,investments,offices
238,walabok,games_video,2.0,None,$0,[],[]
1044,Video Game 911,games_video,7.0,Video Game Console Repairs,$0,[],"[{'description': '', 'address1': '411 East Mai..."
988,Saavn,games_video,25.0,South Asian Music Service,$6M,[],"[{'description': 'Corporate Headquarters', 'ad..."
161,Camlink,games_video,4.0,Online Speed Dating,$0,[],"[{'description': '', 'address1': '', 'address2..."
1080,Fictionaut,games_video,2.0,A literary community,$0,[],"[{'description': 'New York', 'address1': '32-1..."


In [11]:
data.columns

Index(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username', 'category_code',
       'number_of_employees', 'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'email_address', 'phone_number',
       'description', 'created_at', 'updated_at', 'overview', 'image',
       'products', 'relationships', 'competitions', 'providerships',
       'total_money_raised', 'funding_rounds', 'investments', 'offices',
       'video_embeds', 'screenshots', 'external_links', 'partners', 'ipo'],
      dtype='object')

In [12]:
def dropcolumn(df,column):
    df.drop(f"{column}", axis = 1, inplace = True)
    
    return df

In [14]:
columns=['permalink', 'crunchbase_url', 'homepage_url',
       'blog_url', 'blog_feed_url', 'twitter_username',
        'founded_year', 'founded_month', 'founded_day',
       'deadpooled_year', 'deadpooled_month', 'deadpooled_day',
       'deadpooled_url', 'tag_list', 'email_address', 'phone_number',
       'created_at', 'updated_at', 'image',
        'relationships', 'providerships', 'funding_rounds',
       'video_embeds', 'screenshots', 'external_links', 'partners', 'ipo','competitions','overview','_id','products']
for i in columns:
    dropcolumn(data,i)

KeyError: "['permalink'] not found in axis"

In [16]:
dropcolumn(data,'investments')

,name,category_code,number_of_employees,description,total_money_raised,offices
0,Flektor,games_video,NaN,None,$0,"[{'description': None, 'address1': '8536 Natio..."
1,Lala,games_video,NaN,hub for online music discovery,$44.2M,"[{'description': 'Lala Headquarters', 'address..."
2,Joost,games_video,0.0,None,$45M,"[{'description': '', 'address1': '100 5th Ave ..."
3,Babelgum,games_video,NaN,,$13.2M,"[{'description': '', 'address1': '', 'address2..."
4,Sparter,games_video,NaN,,$0,"[{'description': None, 'address1': None, 'addr..."
...,...,...,...,...,...,...
1078,Symposium Productions,games_video,NaN,music entertainment,$0,[]
1079,Rev Media,games_video,NaN,real-time concert viewing,$0,"[{'description': 'HQ', 'address1': '', 'addres..."
1080,Fictionaut,games_video,2.0,A literary community,$0,"[{'description': 'New York', 'address1': '32-1..."
1081,Tiny Speck,games_video,45.0,,$17.2M,"[{'description': '', 'address1': '163 Hastings..."


In [17]:
data['description'] = data['description'].str.upper()

In [18]:
data['description'] = data['description'].str.extract('(.*GAM.*)')

In [ ]:
data.sample(10)

In [19]:
 data=data.dropna(subset=['description'])

In [ ]:
check=data['total_money_raised'].value_counts()
print(check.to_string())

In [20]:
 data=data.dropna(subset=['total_money_raised'])

In [21]:
data=data.explode('offices')

In [22]:
data

,name,category_code,number_of_employees,description,total_money_raised,offices
22,RockYou,games_video,106.0,SOCIAL GAMING COMPANY,$136M,"{'description': '', 'address1': '585 Broadway'..."
26,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,"{'description': 'San Francisco', 'address1': '..."
26,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,"{'description': 'Huntsville', 'address1': '150..."
46,Kongregate,games_video,20.0,GAMING SOCIAL NETWORK,$19M,"{'description': '', 'address1': '2 Shaw Alley'..."
58,MocoSpace,games_video,25.0,LARGEST MOBILE GAMES COMMUNITY IN N AMER,$10.5M,"{'description': '', 'address1': '', 'address2'..."
...,...,...,...,...,...,...
1074,G5 Entertainment,games_video,107.0,PUBLISHER AND DEVELOPER OF MOBILE GAMES,$0,"{'description': 'Development Office', 'address..."
1074,G5 Entertainment,games_video,107.0,PUBLISHER AND DEVELOPER OF MOBILE GAMES,$0,"{'description': 'Headquarters', 'address1': 'G..."
1074,G5 Entertainment,games_video,107.0,PUBLISHER AND DEVELOPER OF MOBILE GAMES,$0,"{'description': 'US Marketing Office', 'addres..."
1075,FragBrag,games_video,4.0,VIDEO SHARING FOR GAMERS,$0,NaN


In [23]:
Test=data['offices'].apply(pd.Series)

In [24]:
Test

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,0
22,,585 Broadway,Suite A,94063,Redwood City,CA,USA,37.484619,-122.206893,NaN
26,San Francisco,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,NaN
26,Huntsville,150 West Park Loop NW,,35806,Huntsville,AL,USA,NaN,NaN,NaN
46,,2 Shaw Alley,4th Floor,94105,San Francisco,CA,USA,37.786942,-122.401245,NaN
58,,,,02111,Boston,MA,USA,42.350274,-71.058768,NaN
...,...,...,...,...,...,...,...,...,...,...
1074,Development Office,"Moskovskiy prospekt, 179-B",,61037,Kharkov,None,UKR,NaN,NaN,NaN
1074,Headquarters,Grev Turegatan 14,,10247,Stockholm,None,SWE,NaN,NaN,NaN
1074,US Marketing Office,"Suite 136, 1750 Montgomery Street",,94111,San Francisco,CA,USA,NaN,NaN,NaN
1075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
dropcolumn(Test,'description')

,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,0
22,585 Broadway,Suite A,94063,Redwood City,CA,USA,37.484619,-122.206893,NaN
26,60 Broadway,,94111,San Francisco,CA,USA,37.787092,-122.399972,NaN
26,150 West Park Loop NW,,35806,Huntsville,AL,USA,NaN,NaN,NaN
46,2 Shaw Alley,4th Floor,94105,San Francisco,CA,USA,37.786942,-122.401245,NaN
58,,,02111,Boston,MA,USA,42.350274,-71.058768,NaN
...,...,...,...,...,...,...,...,...,...
1074,"Moskovskiy prospekt, 179-B",,61037,Kharkov,None,UKR,NaN,NaN,NaN
1074,Grev Turegatan 14,,10247,Stockholm,None,SWE,NaN,NaN,NaN
1074,"Suite 136, 1750 Montgomery Street",,94111,San Francisco,CA,USA,NaN,NaN,NaN
1075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
Test1=pd.concat([data, Test], axis=1)

In [30]:
Test1.sample()

,name,category_code,number_of_employees,description,total_money_raised,offices,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,0
65,OMGPOP,games_video,50.0,FREE ONLINE MULTIPLAYER GAME,$16.6M,"{'description': '', 'address1': 'SoHo', 'addre...",SoHo,,,New York,NY,USA,40.723384,-74.001704,NaN


In [31]:
columns=['state_code','offices','address2','zip_code','address1']
for i in columns:
    dropcolumn(Test1,i)

In [32]:
Test1 = Test1.iloc[:, :-1]

In [ ]:
def dropcolumn(df,column):
    df.drop(f"{column}", axis = 1, inplace = True)

In [ ]:
def dropna(df,column):
    df=df.dropna(subset=[f'{column}'])
    return df
    

In [33]:
Test1=Test1.dropna(subset=['latitude'])

In [34]:
Test1=Test1.dropna(subset=['longitude'])

In [35]:
Test1=Test1.dropna(subset=['number_of_employees'])

In [37]:
Test1=Test1.dropna(subset=['city'])

In [ ]:
check=Test['city'].value_counts()
print(check.to_string())

In [38]:
Test1.reset_index(drop=True, inplace=True)

In [39]:
Test1

,name,category_code,number_of_employees,description,total_money_raised,city,country_code,latitude,longitude
0,RockYou,games_video,106.0,SOCIAL GAMING COMPANY,$136M,Redwood City,USA,37.484619,-122.206893
1,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,San Francisco,USA,37.787092,-122.399972
2,Kongregate,games_video,20.0,GAMING SOCIAL NETWORK,$19M,San Francisco,USA,37.786942,-122.401245
3,MocoSpace,games_video,25.0,LARGEST MOBILE GAMES COMMUNITY IN N AMER,$10.5M,Boston,USA,42.350274,-71.058768
4,OMGPOP,games_video,50.0,FREE ONLINE MULTIPLAYER GAME,$16.6M,New York,USA,40.723384,-74.001704
...,...,...,...,...,...,...,...,...,...
61,Bigpoint,games_video,500.0,DEVELOPER AND PROVIDER OF ONLINE GAMES,€420M,Hamburg,DEU,53.597305,9.976252
62,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,New York,USA,40.752380,-74.005568
63,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,San Francisco,USA,37.787646,-122.402759
64,Video Game 911,games_video,7.0,VIDEO GAME CONSOLE REPAIRS,$0,Denville,USA,40.878656,-74.480020


In [40]:
T=Test1.sort_values(by=["city"]) 

In [ ]:
sns.set_context("poster")
sns.set(rc={"figure.figsize": (16.,6.)})
sns.set_style("whitegrid")    

In [ ]:
sns.countplot(x=T.city)

In [ ]:
T['city'].value_counts()

In [41]:
T['city'] = T['city'].str.strip()

In [42]:
T['city'] = T['city'].astype(str)

In [43]:
SF=T[T["city"] =='San Francisco']

In [44]:
SF

,name,category_code,number_of_employees,description,total_money_raised,city,country_code,latitude,longitude
29,Shattered Reality Interactive,games_video,10.0,"ONLINE SOCIAL GAMING, MMO",$1.1M,San Francisco,USA,37.783046,-122.394067
63,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,San Francisco,USA,37.787646,-122.402759
1,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,San Francisco,USA,37.787092,-122.399972
2,Kongregate,games_video,20.0,GAMING SOCIAL NETWORK,$19M,San Francisco,USA,37.786942,-122.401245
6,Rupture,games_video,25.0,GAMING SOCIAL NETWORK,$3M,San Francisco,USA,37.783898,-122.395234
7,Zynga,games_video,115.0,NETWORK OF GAMING APPLICATIONS,$860M,San Francisco,USA,37.765158,-122.404234
20,Serious Business,games_video,22.0,SOCIAL GAME DEVELOPMENT STUDIO,$4M,San Francisco,USA,37.789321,-122.401362
17,stickychicken,games_video,2.0,PICK-UP GAMES,$0,San Francisco,USA,37.775196,-122.419204
21,Kabam,games_video,650.0,LEADER IN GAMING,$125M,San Francisco,USA,37.394456,-122.078531
27,ngmoco,games_video,150.0,GAMES FOR THE IPHONE,$44.8M,San Francisco,USA,37.770510,-122.401581


In [45]:
T.reset_index(drop=True, inplace=True)

In [ ]:
T

In [46]:
SF.reset_index(drop=True, inplace=True)

In [54]:
SF

,name,category_code,number_of_employees,description,total_money_raised,city,country_code,latitude,longitude
0,Shattered Reality Interactive,games_video,10.0,"ONLINE SOCIAL GAMING, MMO",$1.1M,San Francisco,USA,37.783046,-122.394067
1,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,San Francisco,USA,37.787646,-122.402759
2,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,San Francisco,USA,37.787092,-122.399972
3,Kongregate,games_video,20.0,GAMING SOCIAL NETWORK,$19M,San Francisco,USA,37.786942,-122.401245
4,Rupture,games_video,25.0,GAMING SOCIAL NETWORK,$3M,San Francisco,USA,37.783898,-122.395234
5,Zynga,games_video,115.0,NETWORK OF GAMING APPLICATIONS,$860M,San Francisco,USA,37.765158,-122.404234
6,Serious Business,games_video,22.0,SOCIAL GAME DEVELOPMENT STUDIO,$4M,San Francisco,USA,37.789321,-122.401362
8,Kabam,games_video,650.0,LEADER IN GAMING,$125M,San Francisco,USA,37.394456,-122.078531
9,ngmoco,games_video,150.0,GAMES FOR THE IPHONE,$44.8M,San Francisco,USA,37.770510,-122.401581
10,Shattered Reality Interactive,games_video,10.0,"ONLINE SOCIAL GAMING, MMO",$1.1M,San Francisco,USA,37.783046,-122.394067


In [47]:
SF['total_money_raised'] = SF['total_money_raised'].str.extract('(.*M|B)')

C:\Users\mataf\AppData\Local\Temp\ipykernel_17104\2401967230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SF['total_money_raised'] = SF['total_money_raised'].str.extract('(.*M|B)')


In [ ]:
SF

In [48]:
SF=SF.dropna(subset=['total_money_raised'])

In [ ]:
SF

In [49]:
token='fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo='

In [50]:
load_dotenv()

True

In [51]:
token_fsq = os.getenv("token")

In [52]:
query = "restaurante"

In [53]:
where='SanFrancisco, USA'

In [ ]:
headers = {
    "accept": "application/json",
    "Authorization": token_fsq}

In [ ]:
url_geocode = f"https://geocode.xyz/{where}?json=1"

In [ ]:
res = requests.get(url_geocode).json()

In [ ]:
res

In [ ]:
SF_lon, SF_lat = res["longt"], res["latt"]

In [ ]:
def get_coordinates (place):
    url_geocode = f"https://geocode.xyz/{where}?json=1"

    try:
        res = requests.get(url_geocode).json()
        return res["longt"], res["latt"]
    except:
        print(f"Sorry, no matches found for {place}")
        get_coordinates (place)

In [ ]:
get_coordinates('San Francisco, US')

In [ ]:
url = f"https://api.foursquare.com/v3/places/search?query={query}ll=37.783046%2C-122.394067"

headers = {
    "accept": "application/json",
    "Authorization": token_fsq
}

response = requests.get(url, headers=headers).json()

In [ ]:
url = "https://api.foursquare.com/v3/places/search?query={query}&ll=37.783046%2C-122.394067"

headers = {
    "accept": "application/json",
    "Authorization": token_fsq
}

response = requests.get(url, headers=headers).json()



In [ ]:
response

In [ ]:
import requests

url = "https://api.foursquare.com/v3/places/search?query=restaurante&ll=37.783046%2C-122.394067&radius=800"

headers = {
    "accept": "application/json",
    "Authorization": token_fsq
}

response = requests.get(url, headers=headers).json()


In [55]:
SF

,name,category_code,number_of_employees,description,total_money_raised,city,country_code,latitude,longitude
0,Shattered Reality Interactive,games_video,10.0,"ONLINE SOCIAL GAMING, MMO",$1.1M,San Francisco,USA,37.783046,-122.394067
1,Exent,games_video,100.0,GAMES ON DEMAND LEADER,$3M,San Francisco,USA,37.787646,-122.402759
2,Curse,games_video,58.0,ONLINE GAMES MEDIA COMPANY,$12M,San Francisco,USA,37.787092,-122.399972
3,Kongregate,games_video,20.0,GAMING SOCIAL NETWORK,$19M,San Francisco,USA,37.786942,-122.401245
4,Rupture,games_video,25.0,GAMING SOCIAL NETWORK,$3M,San Francisco,USA,37.783898,-122.395234
5,Zynga,games_video,115.0,NETWORK OF GAMING APPLICATIONS,$860M,San Francisco,USA,37.765158,-122.404234
6,Serious Business,games_video,22.0,SOCIAL GAME DEVELOPMENT STUDIO,$4M,San Francisco,USA,37.789321,-122.401362
8,Kabam,games_video,650.0,LEADER IN GAMING,$125M,San Francisco,USA,37.394456,-122.078531
9,ngmoco,games_video,150.0,GAMES FOR THE IPHONE,$44.8M,San Francisco,USA,37.770510,-122.401581
10,Shattered Reality Interactive,games_video,10.0,"ONLINE SOCIAL GAMING, MMO",$1.1M,San Francisco,USA,37.783046,-122.394067


In [108]:
SF.reset_index(drop=True, inplace=True)

In [112]:
SF_sub=SF[['name','longitude','latitude']]

In [192]:
SF_sub

,name,longitude,latitude
0,Shattered Reality Interactive,-122.394067,37.783046
1,Exent,-122.402759,37.787646
2,Curse,-122.399972,37.787092
3,Kongregate,-122.401245,37.786942
4,Rupture,-122.395234,37.783898
5,Zynga,-122.404234,37.765158
6,Serious Business,-122.401362,37.789321
7,Kabam,-122.078531,37.394456
8,ngmoco,-122.401581,37.770510
9,Shattered Reality Interactive,-122.394067,37.783046


In [318]:
map_2 = Map(location=[Shatt_lat, Shatt_lon], zoom_start=15)

In [367]:
map_2

### ironhack = Marker(
   ### location = [41.3977737, 2.188262117],
   ### tooltip = "Ironhack",
 ###  icon=icon

In [320]:
for index, row in SF_sub.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon = "building-o",
            icon_color = "black",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker.add_to(map_2)
    
    


In [201]:
query='Restaurante'

In [202]:


url = "https://api.foursquare.com/v3/places/search?query={query}&ll=37.787646%2C-122.402759&radius=500"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response = requests.get(url, headers=headers).json()


In [203]:
response

{'results': [],
 'context': {'geo_bounds': {'circle': {'center': {'latitude': 37.787646,
     'longitude': -122.402759},
    'radius': 500}}}}

In [275]:


url = "https://api.foursquare.com/v3/places/search?query=restaurante&ll=37.787646%2C-122.402759&radius=500"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response = requests.get(url, headers=headers).json()

In [209]:
response

{'results': [{'fsq_id': '609c54df340bed636e40a98b',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 380,
   'geocodes': {'main': {'latitude': 37.788044, 'longitude': -122.40706}},
   'link': '/v3/places/609c54df340bed636e40a98b',
   'location': {'address': '333 Post St',
    'census_block': '060750117003003',
    'country': 'US',
    'cross_street': 'stockton',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '333 Post St (stockton), San Francisco, CA 94108',
    'locality': 'San Francisco',
    'neighborhood': ['Chinatown'],
    'postcode': '94108',
    'region': 'CA'},
   'name': 'Alma',
   'related_places': {'parent': {'fsq_id': '40bbc700f964a520b1001fe3',
     'name': 'Union Square'}},
   'timezone': 'America/Los_Angeles'},
  {'fsq_id': '5e600fefd0725c00083bab3a',
   'categories': [{'id': 13065,
     'name': 'Rest

In [218]:
response["results"][0]['geocodes']

{'main': {'latitude': 37.788044, 'longitude': -122.40706}}

In [220]:
response["results"][0]['name']

'Alma'

In [223]:
Alma_latitude = response["results"][0]["geocodes"]["main"]["latitude"]
Alma_longitude = response["results"][0]["geocodes"]["main"]["longitude"]
Alma_address = response["results"][0]["location"]["address"]
Alma_name = response["results"][0]["name"]

In [389]:
response

{'results': [{'fsq_id': '609c54df340bed636e40a98b',
   'categories': [{'id': 13065,
     'name': 'Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 380,
   'geocodes': {'main': {'latitude': 37.788044, 'longitude': -122.40706}},
   'link': '/v3/places/609c54df340bed636e40a98b',
   'location': {'address': '333 Post St',
    'census_block': '060750117003003',
    'country': 'US',
    'cross_street': 'stockton',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': '333 Post St (stockton), San Francisco, CA 94108',
    'locality': 'San Francisco',
    'neighborhood': ['Chinatown'],
    'postcode': '94108',
    'region': 'CA'},
   'name': 'Alma',
   'related_places': {'parent': {'fsq_id': '40bbc700f964a520b1001fe3',
     'name': 'Union Square'}},
   'timezone': 'America/Los_Angeles'},
  {'fsq_id': '5e600fefd0725c00083bab3a',
   'categories': [{'id': 13065,
     'name': 'Rest

In [290]:
new_list = []
for i in response["results"]:
    
    name = i["name"]
    address =  i["location"]["formatted_address"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
    
new_list

[{'name': 'Alma',
  'latitude': 37.788044,
  'longitude': -122.40706,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.788044, -122.40706]}}},
 {'name': 'Twitchen',
  'latitude': 37.79104,
  'longitude': -122.403535,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.79104, -122.403535]}}},
 {'name': 'Fogomarket',
  'latitude': 37.784981,
  'longitude': -122.39988,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.784981, -122.39988]}}},
 {'name': 'Fisher Loft',
  'latitude': 37.789743,
  'longitude': -122.407258,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.789743, -122.407258]}}},
 {'name': 'Soma Garden',
  'latitude': 37.788099,
  'longitude': -122.397124,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.788099, -122.397124]}}},
 {'name': 'Cafe Carousel',
  'latitude': 37.787539,
  'longitude': -122.403185,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.787539, -122.403185]}}},
 {'name

In [292]:
Resto_df= pd.DataFrame(new_list)

In [293]:
Resto_df

,name,latitude,longitude,type
0,Alma,37.788044,-122.407060,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Twitchen,37.791040,-122.403535,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Fogomarket,37.784981,-122.399880,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Fisher Loft,37.789743,-122.407258,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Soma Garden,37.788099,-122.397124,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Cafe Carousel,37.787539,-122.403185,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Good Luck Cafe & Deli,37.788634,-122.403455,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Iron Horse Restaurant,37.788227,-122.404014,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,The Dosa Brothers,37.788949,-122.401913,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,The Sports Club,37.786492,-122.404485,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [321]:
for index, row in Resto_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon = "cutlery",
            icon_color = "white",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_rest = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_rest.add_to(map_2)

In [236]:
GeoResto_df = gpd.GeoDataFrame(Resto_df, geometry=gpd.points_from_xy(Resto_df["lon"], Resto_df["lat"]))
GeoResto_df

,name,lat,lon,type,geometry
0,Alma,37.788044,-122.407060,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40706 37.78804)
1,Twitchen,37.791040,-122.403535,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40354 37.79104)
2,Fogomarket,37.784981,-122.399880,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.39988 37.78498)
3,Fisher Loft,37.789743,-122.407258,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40726 37.78974)
4,Soma Garden,37.788099,-122.397124,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.39712 37.78810)
5,Cafe Carousel,37.787539,-122.403185,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40318 37.78754)
6,Good Luck Cafe & Deli,37.788634,-122.403455,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40345 37.78863)
7,Iron Horse Restaurant,37.788227,-122.404014,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40401 37.78823)
8,The Dosa Brothers,37.788949,-122.401913,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40191 37.78895)
9,The Sports Club,37.786492,-122.404485,"{'typepoint': {'type': 'Point', 'coordinates':...",POINT (-122.40448 37.78649)


In [280]:
url_air = "https://api.foursquare.com/v3/places/search?query=airport&ll=37.787646%2C-122.402759"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_air = requests.get(url_air, headers=headers).json()


In [281]:
response_air

{'results': [{'fsq_id': '41059b00f964a520850b1fe3',
   'categories': [{'id': 19040,
     'name': 'International Airport',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/airport_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 19039,
   'geocodes': {'main': {'latitude': 37.624123, 'longitude': -122.395506},
    'roof': {'latitude': 37.624123, 'longitude': -122.395506}},
   'link': '/v3/places/41059b00f964a520850b1fe3',
   'location': {'census_block': '060819843001002',
    'country': 'US',
    'cross_street': 'at Link Rd',
    'dma': 'San Francisco-Oakland-San Jose',
    'formatted_address': 'at Link Rd, San Francisco, CA 94128',
    'locality': 'San Francisco',
    'neighborhood': ['Belle Air South'],
    'po_box': 'P.O. Box 8097',
    'postcode': '94128',
    'region': 'CA'},
   'name': 'San Francisco International Airport (SFO)',
   'related_places': {'children': [{'fsq_id': '4db37642316a3bec52549b82',
      'name': 'Domestic Parking Garage E - Sh

In [282]:
response_air["results"][0]['name']

'San Francisco International Airport (SFO)'

In [284]:
Int_air_latitude = response_air["results"][0]["geocodes"]["main"]["latitude"]
Int_air_longitude = response_air["results"][0]["geocodes"]["main"]["longitude"]
Int_air_name = response_air["results"][0]["name"]

In [298]:
air_list = []
for i in response_air["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    air_list.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
    
air_list

[{'name': 'San Francisco International Airport (SFO)',
  'latitude': 37.624123,
  'longitude': -122.395506,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.624123, -122.395506]}}},
 {'name': 'San Francisco International Airport (SFO)',
  'latitude': 37.624123,
  'longitude': -122.395506,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.624123, -122.395506]}}},
 {'name': 'AC Hotel by Marriott San Francisco Airport/Oyster Point Waterfront',
  'latitude': 37.664447,
  'longitude': -122.395512,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.664447, -122.395512]}}},
 {'name': 'Oakland International Airport (OAK)',
  'latitude': 37.7117,
  'longitude': -122.212997,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.7117, -122.212997]}}},
 {'name': 'Emirates Sky Cargo',
  'latitude': 37.82484,
  'longitude': -122.314208,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.82484, -122.314208]}}},
 {'name': 'New T-Han

In [299]:
air_df=pd.DataFrame(air_list)

In [312]:
air_df=air_df.loc[(1,3),:]

In [314]:
air_df.reset_index(drop=True, inplace=True)

In [315]:
air_df

,name,latitude,longitude,type
0,San Francisco International Airport (SFO),37.624123,-122.395506,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Oakland International Airport (OAK),37.711700,-122.212997,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [322]:
for index, row in air_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="black",
            opacity = 0.6,
            prefix = "fa",
            icon = "plane",
            icon_color = "white",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_air = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_air.add_to(map_2)

In [328]:
url_vegan = "https://api.foursquare.com/v3/places/search?ll=37.787646%2C-122.402759&categories=13377"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_vegan = requests.get(url_vegan, headers=headers).json()

In [329]:
response_vegan["results"][0]['name']

'Sweetgreen'

In [331]:
new_list_vegan = []
for i in response_vegan["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_vegan.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_vegan

[{'name': 'Sweetgreen',
  'latitude': 37.787182,
  'longitude': -122.398432,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.787182, -122.398432]}}},
 {'name': 'Tender',
  'latitude': 37.786329,
  'longitude': -122.417353,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.786329, -122.417353]}}},
 {'name': 'The Flying Falafel',
  'latitude': 37.78124,
  'longitude': -122.411217,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.78124, -122.411217]}}},
 {'name': 'New Eritrea Restaurant & Bar',
  'latitude': 37.763808,
  'longitude': -122.467787,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.763808, -122.467787]}}},
 {'name': 'Greens Restaurant',
  'latitude': 37.807002,
  'longitude': -122.432295,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.807002, -122.432295]}}},
 {'name': 'Ananda Fuara',
  'latitude': 37.77789,
  'longitude': -122.416164,
  'type': {'typepoint': {'type': 'Point',
    'coordinates':

In [332]:
vegan_df=pd.DataFrame(new_list_vegan)

In [333]:
vegan_df

,name,latitude,longitude,type
0,Sweetgreen,37.787182,-122.398432,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Tender,37.786329,-122.417353,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,The Flying Falafel,37.781240,-122.411217,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,New Eritrea Restaurant & Bar,37.763808,-122.467787,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Greens Restaurant,37.807002,-122.432295,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Ananda Fuara,37.777890,-122.416164,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Papalote Mexican Grill,37.751949,-122.420867,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Beloved Cafe,37.752329,-122.419313,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Pena Pachamama,37.799682,-122.410569,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,Ike's Sandwiches,37.785120,-122.419889,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [336]:
for index, row in vegan_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon = "apple",
            icon_color = "green",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_vegan = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_vegan.add_to(map_2)

In [338]:
url_star = "https://api.foursquare.com/v3/places/search?query=starbucks&ll=37.787646%2C-122.402759"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_star = requests.get(url_star, headers=headers).json()



In [339]:
new_list_star = []
for i in response_star["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_star.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_star

[{'name': 'Starbucks',
  'latitude': 37.788371,
  'longitude': -122.393555,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.788371, -122.393555]}}},
 {'name': 'Starbucks',
  'latitude': 37.793818,
  'longitude': -122.396678,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.793818, -122.396678]}}},
 {'name': 'Starbucks',
  'latitude': 37.78447,
  'longitude': -122.404008,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.78447, -122.404008]}}},
 {'name': 'Starbucks',
  'latitude': 37.789098,
  'longitude': -122.408438,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.789098, -122.408438]}}},
 {'name': 'Starbucks',
  'latitude': 37.789293,
  'longitude': -122.406729,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.789293, -122.406729]}}},
 {'name': 'Starbucks',
  'latitude': 37.808309,
  'longitude': -122.413086,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.808309, -122.413086]}}},
 {'nam

In [340]:
star_df=pd.DataFrame(new_list_star)

In [341]:
star_df

,name,latitude,longitude,type
0,Starbucks,37.788371,-122.393555,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Starbucks,37.793818,-122.396678,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Starbucks,37.784470,-122.404008,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Starbucks,37.789098,-122.408438,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Starbucks,37.789293,-122.406729,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Starbucks,37.808309,-122.413086,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Starbucks,37.786534,-122.408216,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Starbucks,37.787664,-122.401113,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Starbucks,37.790652,-122.403869,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,Starbucks,37.790448,-122.405722,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [342]:
for index, row in star_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon = "coffee",
            icon_color = "white",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_star = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_star.add_to(map_2)

In [344]:
url_kinder = "https://api.foursquare.com/v3/places/search?ll=37.787646%2C-122.402759&categories=12056"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_kinder = requests.get(url_kinder, headers=headers).json()



In [348]:
new_list_kinder = []
for i in response_kinder["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_kinder.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_kinder

[{'name': 'Montessori House Of Children',
  'latitude': 37.785183,
  'longitude': -122.423333,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.785183, -122.423333]}}},
 {'name': 'Presidio Knolls School',
  'latitude': 37.773306,
  'longitude': -122.413743,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.773306, -122.413743]}}},
 {'name': 'LePort School',
  'latitude': 37.776344,
  'longitude': -122.418868,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.776344, -122.418868]}}},
 {'name': 'Preschool Learning Center-Phoebe Hearst',
  'latitude': 37.782779,
  'longitude': -122.42785,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.782779, -122.42785]}}},
 {'name': 'The Little School',
  'latitude': 37.785089,
  'longitude': -122.444615,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.785089, -122.444615]}}},
 {'name': 'Rosenberg Early Childhood Center of the Jccsf',
  'latitude': 37.784703,
  'longitude':

In [349]:
kinder_df=pd.DataFrame(new_list_kinder)

In [350]:
kinder_df

,name,latitude,longitude,type
0,Montessori House Of Children,37.785183,-122.423333,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Presidio Knolls School,37.773306,-122.413743,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,LePort School,37.776344,-122.418868,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Preschool Learning Center-Phoebe Hearst,37.782779,-122.427850,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,The Little School,37.785089,-122.444615,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Rosenberg Early Childhood Center of the Jccsf,37.784703,-122.459428,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Argonne Early Education School,37.774025,-122.474271,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,The Scandinavian School In San Francisco,37.746681,-122.458099,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,Little Bear School,37.723416,-122.438022,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,Playmates Cooperative Nursery School,37.743159,-122.500079,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [353]:
for index, row in kinder_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="beige",
            opacity = 0.6,
            prefix = "fa",
            icon = "child",
            icon_color = "black",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_kinder = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_kinder.add_to(map_2)

In [355]:
url_basket = "https://api.foursquare.com/v3/places/search?ll=37.787646%2C-122.402759&categories=18008"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_basket = requests.get(url_basket, headers=headers).json()

In [356]:
new_list_basket = []
for i in response_basket["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_basket.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_basket

[{'name': 'Berry Basketball Courts',
  'latitude': 37.772031,
  'longitude': -122.397898,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.772031, -122.397898]}}},
 {'name': 'Soma Rec Center',
  'latitude': 37.778838,
  'longitude': -122.406328,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.778838, -122.406328]}}},
 {'name': 'Arterra Mission Bay',
  'latitude': 37.771896,
  'longitude': -122.397532,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.771896, -122.397532]}}},
 {'name': 'Alice Marble Basketball Court',
  'latitude': 37.801395,
  'longitude': -122.420205,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.801395, -122.420205]}}},
 {'name': 'Hamilton Basketball Courts',
  'latitude': 37.784569,
  'longitude': -122.435625,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.784569, -122.435625]}}},
 {'name': 'NBA Nation',
  'latitude': 37.759168,
  'longitude': -122.412335,
  'type': {'typepoint': {'

In [357]:
basket_df=pd.DataFrame(new_list_basket)

In [360]:
for index, row in basket_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="black",
            opacity = 0.6,
            prefix = "fa",
            icon = "futbol-o",
            icon_color = "orange",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_basket = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_basket.add_to(map_2)

In [363]:
url_dog = "https://api.foursquare.com/v3/places/search?ll=37.787646%2C-122.402759&categories=11134"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_dog = requests.get(url_dog, headers=headers).json()


In [364]:
new_list_dog = []
for i in response_dog["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_dog.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_dog

[{'name': 'Wag Hotels San Francisco',
  'latitude': 37.768475,
  'longitude': -122.414103,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.768475, -122.414103]}}},
 {'name': 'Pet Food Express',
  'latitude': 37.78757,
  'longitude': -122.446339,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.78757, -122.446339]}}},
 {'name': 'Pawtrero Hill BathHouse & Feed',
  'latitude': 37.78382,
  'longitude': -122.389322,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.78382, -122.389322]}}},
 {'name': 'Bernal Beast',
  'latitude': 37.739166,
  'longitude': -122.416243,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.739166, -122.416243]}}},
 {'name': 'Bow Wow Meow',
  'latitude': 37.796544,
  'longitude': -122.421711,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.796544, -122.421711]}}},
 {'name': 'The Dog Barber',
  'latitude': 37.770428,
  'longitude': -122.424897,
  'type': {'typepoint': {'type': 'Point',
  

In [365]:
dog_df=pd.DataFrame(new_list_dog)

In [366]:
dog_df=pd.DataFrame(new_list_dog)for index, row in dog_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon = "paw",
            icon_color = "black",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_dog = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_dog.add_to(map_2)

In [380]:
url_night = "https://api.foursquare.com/v3/places/search?ll=37.787646%2C-122.402759&categories=10032"

headers = {
    "accept": "application/json",
    "Authorization": "fsq3EdDQD81IVcvCg3+snCGriZRWovWDJRU0C+/hN2WgRIo="
}

response_night= requests.get(url_night, headers=headers).json()

In [381]:
new_list_night = []
for i in response_night["results"]:
    
    name = i["name"]
    lat = i["geocodes"]["main"]["latitude"]
    lon = i["geocodes"]["main"]["longitude"]
        
    type_ = {"typepoint": 
                          {"type": "Point", 
                           "coordinates": [lat, lon]}}
    
    new_list_night.append({"name":name, "latitude":lat, "longitude":lon, "type":type_})
new_list_night

[{'name': 'Oasis',
  'latitude': 37.771782,
  'longitude': -122.414442,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.771782, -122.414442]}}},
 {'name': 'Public Works',
  'latitude': 37.768956,
  'longitude': -122.419363,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.768956, -122.419363]}}},
 {'name': 'The Great Northern',
  'latitude': 37.767481,
  'longitude': -122.40659,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.767481, -122.40659]}}},
 {'name': 'Evil Eye',
  'latitude': 37.749944,
  'longitude': -122.418026,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.749944, -122.418026]}}},
 {'name': "Jolene's Bar",
  'latitude': 37.765554,
  'longitude': -122.413429,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.765554, -122.413429]}}},
 {'name': 'Club Six',
  'latitude': 37.781406,
  'longitude': -122.40949,
  'type': {'typepoint': {'type': 'Point',
    'coordinates': [37.781406, -122.40949]}}},

In [387]:
night_df=pd.DataFrame(new_list_night)
for index, row in night_df.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["latitude"], row["longitude"]], "tooltip": row["name"]}
    icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon = "moon-o",
            icon_color = "yellow",
            icon_size=(30, 30)
        )
    #3. Marker
    new_marker_night = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    new_marker_night.add_to(map_2)

In [388]:
map_2


In [407]:
import geopy.distance



279352.90160430095


In [450]:
Exent_distance=[]
for index, row in Resto_df.iterrows():
 
    coords_2 = (row["latitude"], row["longitude"])
    Exent_distance.append(geopy.distance.distance(coords_1, coords_2).m)
   
    
    

In [452]:
Exent_distance.sort()

In [453]:
Exent_distance

[39.358816459281,
 125.63426113533987,
 127.98059180186974,
 162.69297441037457,
 198.79844300961915,
 381.4198704801304,
 382.85947372786774,
 389.6259494944495,
 459.58505161480497,
 498.8982682669944]

In [442]:
Exent_lat=SF_sub['latitude'][1]

In [443]:
Exent_long=SF_sub['longitude'][1]

In [444]:
coords_1=Exent_lat,Exent_long

In [445]:
coords_1

(37.787646, -122.402759)

In [ ]:
print(geopy.distance.distance(coords_1, coords_2).m)

In [430]:
coord

{'The Saloon': 'GotGame', 'location': [37.77772, -122.395785]}

In [421]:
Shatter

'Shattered Reality Interactive'

In [423]:
SF_sub

,name,longitude,latitude
0,Shattered Reality Interactive,-122.394067,37.783046
1,Exent,-122.402759,37.787646
2,Curse,-122.399972,37.787092
3,Kongregate,-122.401245,37.786942
4,Rupture,-122.395234,37.783898
5,Zynga,-122.404234,37.765158
6,Serious Business,-122.401362,37.789321
7,Kabam,-122.078531,37.394456
8,ngmoco,-122.401581,37.770510
9,Shattered Reality Interactive,-122.394067,37.783046


In [446]:
Resto_df


,name,latitude,longitude,type
0,Alma,37.788044,-122.407060,"{'typepoint': {'type': 'Point', 'coordinates':..."
1,Twitchen,37.791040,-122.403535,"{'typepoint': {'type': 'Point', 'coordinates':..."
2,Fogomarket,37.784981,-122.399880,"{'typepoint': {'type': 'Point', 'coordinates':..."
3,Fisher Loft,37.789743,-122.407258,"{'typepoint': {'type': 'Point', 'coordinates':..."
4,Soma Garden,37.788099,-122.397124,"{'typepoint': {'type': 'Point', 'coordinates':..."
5,Cafe Carousel,37.787539,-122.403185,"{'typepoint': {'type': 'Point', 'coordinates':..."
6,Good Luck Cafe & Deli,37.788634,-122.403455,"{'typepoint': {'type': 'Point', 'coordinates':..."
7,Iron Horse Restaurant,37.788227,-122.404014,"{'typepoint': {'type': 'Point', 'coordinates':..."
8,The Dosa Brothers,37.788949,-122.401913,"{'typepoint': {'type': 'Point', 'coordinates':..."
9,The Sports Club,37.786492,-122.404485,"{'typepoint': {'type': 'Point', 'coordinates':..."


In [ ]:
data['total_money_raised'] = data['total_money_raised'].str.extract('(.*M|B)')

In [ ]:
#col=col.dropna(subset=['latitude'])

In [ ]:
#columna=Test['offices'].apply(lambda x:x['latitude'])

In [ ]:
#y=json.loads('Offices.json')